<center><h2><strong><font color="blue">WFH 2024 idBigData - Loading LLM Merak & Prompting</font></strong></h2></center>

<img alt="" src="images/covers/cover_taudata_uin.jpg"/>

# Jangan lupa mengganti Runtime menjadi GPU di Google Colab

## Sumber: https://huggingface.co/Ichsan2895/Merak-7B-v1 

### Serupa untuk LLM (non-API & Open Source) yang lain.

In [ ]:
import warnings; warnings.simplefilter('ignore') 

try:
    import google.colab; IN_COLAB = True
    print("Installing the required modules")
    !pip install protobuf==4.24.4 --q
    !pip install bitsandbytes==0.41.1 --q
    !pip install transformers==4.34.1 --q
    !pip install peft==0.5.0 --q
    !pip install accelerate==0.26.0 --q
    !pip install einops==0.6.1 scipy sentencepiece datasets --q
    !pip install transformers peft datasets --q
    print("preparing directories and assets")
    !mkdir data images output models
    #!wget https://raw.githubusercontent.com/taudata...
except:
    IN_COLAB = False
    print("Running the code locally, please make sure all the python module versions agree with colab environment and all data/assets downloaded")

In [ ]:
import torch, numpy as np
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from peft import PeftModel, PeftConfig

seed = 0
np.random.seed(seed)
"Done"

In [ ]:
model_id = "Ichsan2895/Merak-7B-v4"
config = AutoConfig.from_pretrained(model_id)

BNB_CONFIG = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_compute_dtype=torch.bfloat16,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",)

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=BNB_CONFIG,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = LlamaTokenizer.from_pretrained(model_id)
"Done"

In [ ]:
def generate_response(question: str) -> str:
    chat = [
      {"role": "system", "content": "Anda adalah Merak, sebuah model kecerdasan buatan yang dilatih oleh Muhammad Ichsan. Mohon jawab pertanyaan berikut dengan benar, faktual, dan ramah."},
      {"role": "user", "content": question},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)

    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"),
                           attention_mask=inputs.attention_mask,
                           eos_token_id=tokenizer.eos_token_id,
                           pad_token_id=tokenizer.eos_token_id,
                           max_new_tokens=256)
        response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

        assistant_start = f'''{question} \n assistant\n '''
        response_start = response.find(assistant_start)
        return response[response_start + len(assistant_start) :].strip()

"Done"

In [ ]:
prompt = "Siapa presiden pertama dan kelima Indonesia?"
print(generate_response(prompt))

In [ ]:
prompt = "Apa Ibukota Jawa Timur?"
print(generate_response(prompt))

In [ ]:
prompt = "Apa Ibukota Cililitan?"
print(generate_response(prompt)) # Halusinasi

In [ ]:
prompt = "Buatkan saya pantun yang lucu tentang AI"
print(generate_response(prompt))

In [ ]:
prompt = "jika Budi memiliki gaji bulanan 12 juta dan pajak tahunan pemerintah 10% berapa penghasilan bersih Budi tiap bulan?"
print(generate_response(prompt)) # karena Prompt-nya lebih kompleks response Merak lebih lama. Tidak di desain untuk Logika Matematis.